In [1]:
from pydriller import Repository
from pydriller.domain.commit import ModificationType, Commit
import pandas as pd

In [2]:
repository_path = "hadoop"

In [3]:
def get_repository(path):
    return Repository(path,)

In [4]:
repository = get_repository(repository_path)
print("successfully loaded repository")

successfully loaded repository


In [5]:
commits = repository.traverse_commits()

In [6]:
def get_keys_list():
    df = pd.read_excel("Issues_assignment1.xlsx")

    # Extract the keys from the 'key' column
    keys = df['Key'].tolist()
    
    return keys

In [7]:
def is_issue_present(msg: str):
    keys = get_keys_list()
    
    for key in keys:
        if msg.__contains__(key):
            return True
        
    return False
        

In [8]:
def analyze_commit(data: Commit):
    # Initialize counters
    added_files = modified_files = deleted_files = 0
    added_lines = deleted_lines = 0
    added_methods = deleted_methods = changed_methods = 0
    total_complexity = 0
    
    print(f"Commit hash: {data.hash}")
    print(f"Author: {data.author.name} <{data.author.email}>")
    print(f"Date: {data.committer_date}")
    print(f"Message: {data.msg}")
    
    # Analyze modifications in each commit
    for modified_file in data.modified_files:
        if modified_file.change_type == ModificationType.ADD:
            added_files += 1
        elif modified_file.change_type == ModificationType.MODIFY:
            modified_files += 1
        elif modified_file.change_type == ModificationType.DELETE:
            deleted_files += 1

        added_lines += modified_file.added_lines
        deleted_lines += modified_file.deleted_lines

        # if modified_file.filename.endswith('.java'):
        #     with open(modified_file.new_path, 'r', encoding="utf-8", errors='ignore') as file:
        #         content = file.read()
        #     try:
        #         complexity = cc_visit(content)
        #         total_complexity += sum([node.complexity for node in complexity])
        # 
        #         added_methods += len([m for m in complexity if m.name and m.new])
        #         deleted_methods += len([m for m in complexity if m.name and m.old])
        #         changed_methods += len([m for m in complexity if m.name and m.diff])
        #     except Exception as e:
        #         print(f"Error processing file {modified_file.filename}: {e}")

    print(f"No. of added files: {added_files}")
    print(f"No. of modified files: {modified_files}")
    print(f"No. of deleted files: {deleted_files}")

    print(f"No. of added lines of code: {added_lines}")
    print(f"No. of deleted lines of code: {deleted_lines}")

    print(f"No. of added methods: {added_methods}")
    print(f"No. of deleted methods: {deleted_methods}")
    print(f"No. of changed methods: {changed_methods}")

    print(f"Total cyclomatic complexity: {total_complexity}")
    print()

In [9]:
for commit in commits:
    if is_issue_present(commit.msg):
        analyze_commit(commit)

Commit hash: 6ee5a73e0e91a2ef27753a32c576835e951d8119
Author: Thomas White <tomwhite@apache.org>
Date: 2011-08-19 17:26:42+00:00
Message: HADOOP-7264. Bump avro version to at least 1.4.1. Contributed by Alejandro Abdelnur

git-svn-id: https://svn.apache.org/repos/asf/hadoop/common/trunk@1159699 13f79535-47bb-0310-9956-ffa450edef68
No. of added files: 0
No. of modified files: 9
No. of deleted files: 0
No. of added lines of code: 57
No. of deleted lines of code: 44
No. of added methods: 0
No. of deleted methods: 0
No. of changed methods: 0
Total cyclomatic complexity: 0

Commit hash: a2b1aa947105f0a20fcc5aeed302f77a421fe708
Author: Aaron Myers <atm@apache.org>
Date: 2011-09-01 17:13:44+00:00
Message: HADOOP-7595. Upgrade dependency to Avro 1.5.3. (Alejandro Abdelnur via atm)


git-svn-id: https://svn.apache.org/repos/asf/hadoop/common/trunk@1164174 13f79535-47bb-0310-9956-ffa450edef68
No. of added files: 0
No. of modified files: 5
No. of deleted files: 0
No. of added lines of code: 10
No

Exception ignored in: <function ZipFile.__del__ at 0x104151d00>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/zipfile/__init__.py", line 1938, in __del__
    def __del__(self):

KeyboardInterrupt: 


BrokenPipeError: [Errno 32] Broken pipe